# Demo: Loading data

This notebook demonstrates how to load data from one of the configured STAC catalogs available in this package.

The notebook demonstrates

1. Importing the relevant modules and classes
2. How to use the `QueryParams` and `LoadParams` classes to specify loading information
3. How to instantiate an instance of the `RasterBase` class from a STAC query
4. The contents of the `data` and `masks` attributes in the instantiated `RasterBase`
5. How to access various items of metadata from the instantiated `RasterBase`

## Why the RasterBase class?
The idea with this class is to keep Earth observation bands separated from masking bands so that they can be handled independently, but used in combination as needed.

At this time, the class can only be instantiated by loading from a STAC catalog, but the design has been chosen to allow for additional loading methods in future (e.g. from_file, from_dc). 
The benefit to this approach is that the structure used to hold and manage EO data is the same irrespective of its source.

## Set up
The following cell should be uncommented and run if you installed the package in editable mode and are actively developing and testing modules.
Otherwise, it can be left commented.

In [ ]:
# %load_ext autoreload
# %autoreload 2

### Enable logging

This will allow you to see info and warning messages from the package.

In [ ]:
import logging
import sys

logging.basicConfig(
    format="%(asctime)s | %(levelname)s : %(message)s",
    level=logging.INFO,
    stream=sys.stdout,
)

### Import the relevant packages

To load data, this demo imports the three required classes from the `rasterbase` module:

- `RasterBase` - contains data and masks after loading
- `QueryParams` - contains metadata that specifies the area and date range to load data for
- `LoadParams` - contains metadata that specifies the details of the data load, such as CRS, resolution, and bands

This demo uses the Digital Earth Africa Sentinel-2 product, and hence imports the `de_africa_stac_config`.
For more information on available configurations, see [configuration_demo.ipynb](configuration_demo.ipynb)

In [ ]:
from eo_insights.raster_base import RasterBase, QueryParams, LoadParams
from eo_insights.stac_configuration import de_africa_stac_config

## Set up query and load params

These classes have been designed to separate the parameters that specify the area of interest and the date range from those that are related to the loading settings for a given product. 
This would allow a single `QueryParams` instance to be used in multiple loads, each having a unique `LoadParams` instance as needed.

In [ ]:
query_params = QueryParams(
    bbox=(37.76, 12.49, 37.77, 12.50),
    start_date="2020-11-01",
    end_date="2020-12-01",
)

load_params = LoadParams(
    crs="EPSG:6933", resolution=20, bands=("red", "green", "blue", "nir", "scl")
)

## Load using RasterBase

To load from a STAC catalog, `RasterBase` has a `from_stac_query()` method, which runs the following steps:

- Connects to the desired catalog using `pystac_client`
- Searches the catalog for the  STAC items that match the query and load parameters 
- Runs `odc.stac.load()` using the relevant configuration object to provide useful metadata
- Identifies whether any of the loaded bands are masks and moves them to a separate attribute

The result is an instantiated instance of `RasterBase` which has two attributes: `data` and `masks`


In [ ]:
stac_raster = RasterBase.from_stac_query(
    config=de_africa_stac_config,
    collections=["s2_l2a"],
    query_params=query_params,
    load_params=load_params,
)

### Inspect attributes of RasterBase

Both the `data` and `masks` attributes are Xarray Datasets.

In [ ]:
stac_raster.data

In [ ]:
stac_raster.masks

## Accessing metadata from RasterBase

This section demonstrates how to use `RasterBase` in conjunction with `odc.geo` to access various items of metadata from the loaded data.

Since `data` and `masks` are Xarray Datasets, both can be extended with the `.odc` accessor from `odc.geo`. 
For more information, see the [odc-geo documentation](https://odc-geo.readthedocs.io/en/latest/intro-xr.html).

In [ ]:
# CRS
epsg_number = stac_raster.data.odc.crs.epsg
print(f"ESPG number: {epsg_number}")
crs = stac_raster.data.odc.crs.to_wkt()
print(f"WKT CRS: {crs} \n")

# Bounds
extent = stac_raster.data.odc.geobox.extent.boundingbox.bbox
print(f"Data CRS Bounds: {extent}")
bounds = stac_raster.data.odc.map_bounds()
print(f"Lat Lon CRS Bounds: {bounds} \n")

# Resolution
resolution = stac_raster.data.odc.geobox.resolution
print(f"Resolution: {resolution}")
print(f"Resolution in x: {resolution.x}")
print(f"Resolution in y: {resolution.y} \n")

# Shape
height = stac_raster.data.odc.geobox.height
width = stac_raster.data.odc.geobox.width
print(f"Shape: {height, width} \n")

# Transform
transform = stac_raster.data.odc.geobox.transform
print(f"Transform: {transform} \n")

# Band level info
blue_nodata_value = stac_raster.data.blue.nodata
print(f"Blue nodata value: {blue_nodata_value}")
blue_type = stac_raster.data.blue.dtype
print(f"Blue type: {blue_type}")